<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Преобразование последовательностей: 1-к-1 и N-к-M](#toc1_)    
- [Распознавание плоской структуры коротких текстов](#toc2_)    
  - [Постановка задачи](#toc2_1_)    
  - [Схемы кодирования меток](#toc2_2_)    
- [Архитектуры распознавания плоских структур](#toc3_)    
  - [Условные случайные поля. Базовые принципы](#toc3_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Преобразование последовательностей: 1-к-1 и N-к-M](#toc0_)

А что, если на входе - текст, и на выходе - текст? Это работа для переводчика, которому, как мы знаем, контекст важнее всего. Если вы должны перевести один массив текста в другой, или в несколько, то этот модуль даст вам всё необходимое!

# <a id='toc2_'></a>Распознавание [плоской структуры](https://en.wikipedia.org/wiki/Shallow_parsing) коротких текстов[](#toc0_)

Задача заключается в выделении в тексте сегментов или коротких подпоследовательностей и соотнесения их с заданными категориями. Эта задача чем-то похожа на семантическую сегментацию изображений.

На самом деле это целый класс задач. Его еще называют "**chunking**" или "**shallow parsing**" (т.е. поверхностный разбор) Рассмотренный ранее POS-tagging (снятие частеречной омонимии) это задача данного типа. Также сюда входит [распознавание именованных сущностей](https://en.wikipedia.org/wiki/NER):

- **Вход**: короткий текст (чаще всего это отдельное предложение)
- Для каждого токена предсказывается класс (локация, персона, организация, прочее, и т.д.)
- **Выход**: список меток классов по длине входящего списка токенов

Наиболее очевидные применения включают:
- снятие частиречной омонимии, 
- поверхностный синтаксический анализ — когда нужно найти все словосочетания определённого вида (например, существительное и прилагательное или существительное и существительное), но, при этом, не нужно выделять какую-то сложную иерархию. 
- задачи поиска (чтобы искать не только по отдельным словам, но и по более сложным элементам), 
- распознавание именованных сущностей (названий организации, лекарств, фамилий, локаций, и так далее), 
- извлечение фактов (то есть связанных между собой сущностей), 
- сегментация текста, разбиения на токены, предложения (кстати, в некоторых языках даже это может быть проблемой), 
- выделение заголовков в списках литературы. 
 
Короче говоря, уметь **распознавать плоскую структуру** — это практично, полезно и, при этом, не так сложно, как делать, например, полный семантический анализ (структура уже не плоская, а дерево).

## <a id='toc2_1_'></a>[Постановка задачи](#toc0_)

Распознавание плоской структуры - это **задача классификации**, но с особенностями.

Задача классификации:
- Дан набор объектов $X=\{x_i\}$ (множество), для каждого объекта нужно предсказать одну метку $y_i \in Y$
- Данные - независимые и одинаково распределенные случайные величины (i.i.d. - independent and identically-distributed)
- Предсказания независимые $P(y|X) = \prod_i P(y_i|x_i)$ - т.е. мы можем просто брать произведение условных вероятностей.
- Ключевое допущение тут, то что на метку объекта влияет только сам объект, а другие не влияют

Задача распознавания плоской структуры:
- Дан набор объектов $X=\{x_i\}$, каждый объект состоит из набора элементов $x_i = <x_{ij}>_j$ (вектор/кортеж, т.е. упорядоченный набор)
- Для каждого **элемента** нужно предсказать метку $y_i = <y_{ij}>_j, y_{ij} \in Y$
- **Объекты** по прежнему - независимые и одинаково распределенные случайные величины (i.i.d. - independent and identically-distributed), т.е. $P(y|X) = \prod_i P(y_i|x_i)$
- **Элементы объектов не являются независимыми** $P(y_j|x_j) \neq \prod_i P(y_{ij}|x_{ij})$
- Т.е. мы не можем в случайном предложении взять отдельное слово и просто предсказать его класс, его метка будет зависеть еще и от соседних слов.

В основе всех алгоритмов решения задач такого класса лежит поиск хороших и простых моделей / приближений (вычислительно приемлемых и достаточно точных) для сложного распределения:
$$P(y_j|x_j) = P(y_{i1},y_{i2},...,y_{il}|x_{i1},x_{i2},...,x_{il})$$

**Например**

Вы делаете задачу определения частей речи (POS-теггинга).

Вам дано предложение "мама мыла раму", токенизированное как $x_1 = <x_{11}="мама", \quad x_{12}="мыла", \quad x_{13}="раму">$. Каждому токену $x_{ij}$ может быть назначен один из нескольких классов $y_{ij} \in \{NOUN, VERB, PUNCT, ...\}$

Тогда задачу определения частей речи можно поставить как задачу оптимизации

$$y_{i1}, y_{i2}, ..., y_{in} = \argmax_{y_{i1}, y_{i2}, ..., y_{in}} P(y_{i1}, y_{i2}, ..., y_{in} | x_{i1}, x_{i2}, ..., x_{in})$$

Где условная вероятность $P$ определяется как:
$$P(y_{i1}, y_{i2}, ..., y_{in} | x_{i1}, x_{i2}, ..., x_{in}) = \prod_{j=1}^n P(y_{ij} | x_{i1}, x_{i2}, ..., x_{in}, y_{i1}, ..., y_{i(j-1)})$$

- т.е. для правильного определения класса конкретного токена нам важно учитывать и соседние токены и их классы



## <a id='toc2_2_'></a>[Схемы кодирования меток](#toc0_)

Метки зависят от контекста, одни и те же слова в разных ситуациях могут получать разные метки. Это создаёт некоторые сложности, и поэтому применяют специальные схемы — схемы кодирования меток.

**Например**:
Если у нас для каждой сущности только одна метка:
- "Росгидромет передает ..." -> [ORG, ...]
- "Федеральная служба по гидрометеорологии и мониторингу окружающей среды передает ..." -> [ORG,ORG,ORG,ORG,ORG,ORG,ORG,ORG,ORG,...]
- а что, если в последнем примере классификатор неверно определит "и" -> [ORG,ORG,ORG,ORG,**None**,ORG,ORG,ORG,ORG,...]
  -  получим, вместо одной правильной сущности, две очень странные сущности, которые не соответствуют ничему в реальной жизни
- а что, если в предложении идут две сущности подряд без союза или без знака препинания между ним. Как их разделить, если все метки одинаковые?

Проблемы:
- большая зависимость от контекста - разделяющая поверхность усложняется
- чувствительность к шуму
- отсутствие разделения идущих подряд сущностей
  
Нужны дополнительные метки — дополнительные классы, которые мы сами вводим для того, чтобы обрабатывать такие специальные ситуации.

1. **IO-кодирование** "inside-outside" - 1 метка

Самый простой способ — это IO-кодирование, то есть внутри или снаружи. Показан выше. Т.е. если слово входит в наименование сущности, то отмечается тегом, **идущие подряд теги - это одна сущность**.

2. **[BIO](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging))-кодирование** "beginning, inside, outside" - 2 метки

Кодируется начальный элемент и внутренний элемент: [ORG-B,ORG-I,ORG-I,ORG-I,ORG-I,ORG-I,ORG-I,ORG-I,ORG-I,None...].
Cамая популярная схема - простая и её достаточно в большинстве случаев

3. **BMEWO-кодирование** - 4 метки

Кодрируется начало, середина и конец сущности, а также однословные сущности.

\*) можно придумать и более сложные схемы, лишь бы они добавляли информацию в эмбеддинг больше, чем шум  
\**) метка конца сущности ORG-O тут в примерах нигде не используется. Может можно придумать, где бы она была полезна? Обозначить следующий токен, что он не сущность, но идет следом за сущностью? Наверно как то так, должно повысить качество распознавания сущностей.

# <a id='toc3_'></a>[Архитектуры распознавания плоских структур](#toc0_)

Актуальная на сегодня архитектура решения данной задачи (применяется чуть реже, чем всегда) подразумевает последовательное применение к тексту следующих блоков:

1. Слой получения **эмбеддингов** слов или символов
   - дистрибутивно-семантические модели - word2vec, FastText,
   - предобученные языковые модели - BERT, ELMo
2. **Контекстуализация** - добавление в вектор каждого слова информации не только о нём самом, но и о его соседях
    - любые архитуктуры нейронных сетей (сверточные (Conv1d), рекуррентные (RNN), механизмы внимания (Attention), предобученные языковые модели (BERT, ELMo) и т.д.)
    - они могут, если им надо, изменить размерность векторов (EmbSize), но не могут менять длину последовательности (InLen)
3. Отображение контекстуализированных признаков в **пространство вероятностей меток**
   - количество строк в матрице будет InLen, а столбцов - по количеству самих различных меток (зависит как от количества сущностей, так и от способа кодирования меток)
   - тут уже вероятности предстказываются независимо по каждому вектору токена, вся информация о контексте уже в нем
4. **Учет ограничений** на совместное распределение меток
   - исправляются ошибки, например, когда после окончания сущности идёт среднее слово — такого встречаться не должно
   - т.е. выбирается наиболее вероятное сочетание кодов меток
   - чаще всего, здесь используется специальный вид графических вероятностных моделей — это "[**условные случайные поля**](https://en.wikipedia.org/wiki/Conditional_random_field)" или **CRF** (conditional random fields)

\*) Пункт 4 опциональный, необходимость зависит его от конкретного дизайна задачи

## <a id='toc3_1_'></a>[Условные случайные поля. Базовые принципы](#toc0_)

CRF — это целый класс очень мощных графических моделей общего назначения.

$$P(y|x) = P(y_{1},y_{2},...,y_{l}|x_{1},x_{2},...,x_{l})$$

Условные случайные поля, чаще всего, применяются в **задачах сегментации** разного рода — сегментации картинок например, или текстов, когда объекты состоят из множества базовых элементов, для каждого из которых нужно предсказать метку и метки зависят друг от друга. То есть, соответствующее распределение вероятностей этих меток (оценивается по частоте в обучающей выборке, спасибо кэп) не факторизуется. 

Эта модель относится к классу **неориентированных графических моделей**, или **марковских сетей**. 

Если мы решаем задачу классификации, то нам не обязательно полностью моделировать вот это сложное распределение, нам достаточно уметь находить точку его максимума. То есть, уметь находить **наиболее вероятное сочетание меток**. 

$$\hat y_{1}, \hat y_{2}, ..., \hat y_{l} = \argmax_{y_{i}} P(y_{1}, y_{2}, ..., y_{l} | x_{1}, x_{2}, ..., x_{l})$$

Однако и эта задача, в общем случае, является крайне сложной с вычислительной точки зрения и требует численной оптимизации для каждого примера. Это может быть очень дорого на практике. 

Для решения данной задачи на текстах **используется приближение** - текст можно рассматривать как цепочку объектов (вообще, текст - более сложная структура, нежели просто цепочка). В такой постановке метка класса $y_i$ зависит от вектора признаков токена $x_i$, которую получила нейросеть и значений соседних меток классов $y_{i-1}, y_{i+1}$:

    y1 -> y2 -> y3 -> ... ->
    ^     ^     ^  
    |     |     |
    x1    x2    x3

В такой постановке, распределение вероятностей меток уже факторизуется:
$$P(y_{1}, y_{2}, ..., y_{l} | x_{1}, x_{2}, ..., x_{l}) = \prod_{i} P(y_{i} | x_{i}, y_{i-1}, ..., y_{i+1})$$

В случае с такой линейной топологией, существует эффективный алгоритм, основанный на динамическом программировании (forward-backward, Viterby)[1] 

С точки зрения прикладного результата, CRF ограничивает некорректные переходы между метками и снижает уровень шума, то есть он имеет возможность исправить некоторые ошибки классификатора. В целом, качество существенно улучшается.

[1] [Алгоритм Витерби](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%92%D0%B8%D1%82%D0%B5%D1%80%D0%B1%D0%B8)


Семинар в директории `./stepik-dl-nlp`